In [164]:
## importing Necessary libraries 

from bs4 import BeautifulSoup
import requests
import pandas as pd 
import re
import numpy as np

In [165]:
## Creating list for storing 
reviews = []
stars = []
date = []
country = []
travelled_class = []
leisure_type = []
recommended = []
route = []
pattern = r'(Economy Class|Business Class|First Class|Premium Economy)'

In [166]:
## Scarping the data 
for i in range(1,39):
    url = f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100"
    webpage = requests.get(url)
    soup = BeautifulSoup(webpage.content,"lxml")
    # Reviews
    for item in soup.find_all("div", class_="text_content"):
        reviews.append(item.text)
    # Ratings
    for item in soup.find_all("div", class_ = "rating-10"):
        try:
            stars.append(int(item.span.text))
        except:
            continue
    # Travelled Class
    for item in soup.find_all("div", class_="review-stats"):
        try:
            travelled_class.append(re.findall(pattern,item.text)[0])
        except:
            travelled_class.append(np.nan)
    # Date Flown
    for item in soup.find_all("time"):
        date.append(item.text)
    # Country
    for item in soup.find_all("h3"):
        country.append(item.span.next_sibling.text.strip(" ()"))
    # Leisure Type
    for item in soup.find_all('td', class_='review-rating-header type_of_traveller'):
        leisure_type.append(item.find_next_sibling('td', class_='review-value').text)
    # Recommended Type
    for item in soup.find_all('td', class_='review-rating-header recommended'):
        try:
            recommended.append(item.find_next_sibling('td', class_='review-value rating-no').text)
        except:
            recommended.append("Yes")
    # Route Flown
    for item in soup.find_all('td', class_='review-rating-header route'):
        route.append(item.find_next_sibling('td', class_='review-value').text)
    


In [167]:
## Checking If all lengths are equal because then only we can create dataframe
print(len(reviews),len(stars),len(travelled_class),len(date),len(country),len(recommended))


3799 3832 3799 3799 3799 3799


In [168]:
## Stars list has more values so we are reducung it to common scale to create dataframe
stars = stars[:3799]
len(stars)

3799

In [169]:
## Create  a dataframe from these collected lists of data

df = pd.DataFrame({"Date Flown":date,
                   "Reviews":reviews,
                   "Ratings Provided": stars, 
                   "Country": country,
                   "Travelled Class":travelled_class,
                   "Recommended Type":recommended
                   })

In [170]:
df

,Date Flown,Reviews,Ratings Provided,Country,Travelled Class,Recommended Type
0,26th May 2024,✅ Trip Verified | A premium price for a sub-p...,5,United Kingdom,Business Class,no
1,20th May 2024,✅ Trip Verified | Really terrible user experi...,3,United States,Economy Class,no
2,18th May 2024,✅ Trip Verified | Very impressed with BA. Chec...,3,United Kingdom,Economy Class,Yes
3,14th May 2024,"✅ Trip Verified | LHR - SFO, LAS - LGW August...",9,United Kingdom,Business Class,no
4,8th May 2024,Not Verified | I flew from Malaga via LHR to...,3,Canada,Economy Class,no
...,...,...,...,...,...,...
3794,29th August 2012,LHR-JFK-LAX-LHR. Check in was ok apart from be...,8,United Kingdom,Economy Class,no
3795,28th August 2012,LHR to HAM. Purser addresses all club passenge...,9,United Kingdom,Business Class,Yes
3796,12th October 2011,My son who had worked for British Airways urge...,8,United Kingdom,Economy Class,Yes
3797,11th October 2011,London City-New York JFK via Shannon on A318 b...,10,United States,Premium Economy,no


In [171]:
## Exporting the dataframe into csv 
df.to_csv("D:"+ "/British_Airways_Reviews.csv")

In [173]:
df["Travelled Class"].value_counts()

Economy Class      1978
Business Class     1235
Premium Economy     369
First Class         215
Name: Travelled Class, dtype: int64